# Jira How-to

This notebook walks through how to use the jira python package to work with our Jira instance.

## Obtain an API token for Jira

- Go to https://issues.redhat.com and log in.
- Click on your profile icon in the top right corner and select "Profile".
- Click on "Personal Access Tokens" in the left sidebar.
- Click on "Create token"
- Give the token a name, and an expiration date (optional, but recommended)
- Click "Create"
- Copy the token to a safe place. You will not be able to see it again.

NOTE: This token has the same permissions as your account, so keep it safe.

You will need to set it as an environment variable. The easiest way to do this
is to add the following to a file called `.env` in the current directory:

```shell
JIRA_TOKEN=xxxxxxxxxxxxxxxxxxxxxxx
```

----

## Let's play with Atlassian's Jira API...

https://atlassian-python-api.readthedocs.io/index.html

This package is a wrapper around the Jira API, and it isn't properly typed... The calls just return dictionaries.

In [ ]:
# Create a JIRA client
from os import environ
from atlassian import Jira
jira_api_token = environ.get('JIRA_TOKEN', '')
jira_url = environ.get('JIRA_URL', '')
jira = Jira(url=jira_url, token=jira_api_token)

In [ ]:
issue_key = 'RHSTOR-919'
issue = jira.issue(issue_key)
print(f"{type(issue)}\n")

from pprint import pprint
pprint(issue)

In [ ]:
# Let's get the properties as well...
issue_with_properties = jira.get_issue(issue_key, properties="*all")
pprint(issue_with_properties)

In [ ]:
rlinks = jira.get_issue_remote_links(issue_key)
for rlink in rlinks:
    print(f"{rlink['object']['title']} -> {rlink['object']['url']}")

In [ ]:
# Lets look at all the possible Jira fields and their types:
fields = jira.get_all_fields()
for field in sorted(fields, key=lambda x: x['id']):
    type = field['schema']['type'] if 'schema' in field else 'unknown'
    print(f"{field['id']} -> {field['name']} -- {type}")

In [ ]:
for field in fields:
    if field['id'] in ['customfield_12311140', 'customfield_12311141', 'customfield_12313140', 'customfield_12318341']:
        pprint(field)


In [ ]:
# Accessing "Parent Link" custom field
et85 = jira.get_issue('OCTOET-85')
pprint(et85['fields']['customfield_12313140'])  # Has parent

stor919 = jira.get_issue('RHSTOR-919')
pprint(stor919['fields']['customfield_12313140'])  # No parent

In [ ]:
interesting_fields = ['customfield_12311140', 'customfield_12311141', 'customfield_12313140']
issue = jira.get_issue('OPRUN-3254')
for field in interesting_fields:
    field_name = "unknown"
    for idx in fields:
        if idx['id'] == field:
            field_name = idx['name']
            break
    print(f"{field_name} -> {issue['fields'].get(field, None)}")

In [ ]:
# Get a list of the custom fields in this issue by finding all fields that start with "customfield_"
custom_fields = [k for k in issue['fields'].keys() if k.startswith('customfield_')]
for field in custom_fields:
    if issue['fields'].get(field, None) is None:
        continue
    field_name = "unknown"
    id = "?"
    for idx in fields:
        if idx['id'] == field:
            id = idx['id']
            field_name = idx['name']
            break
    print(f"{field_name} ({id}) -> {issue['fields'].get(field, None)}")


In [ ]:
import sys


if 'jhelper' in sys.modules:
    del sys.modules['jhelper']
import jhelper

for i in ['OPRUN-1858', 'OPRUN-3254', 'RHSTOR-919']:
    print(f"Related issues for {i}:")
    issue = jhelper.Issue(jira, i)
    related = issue.related
    for r in related:
        print(f"  {r.how} -> {r.key}")

In [ ]:
pprint(jira.jql("labels = 'AISummary' ORDER BY created DESC", limit=5, fields="key,summary,updated"))

In [ ]:
jira.get_issue_changelog('RHSTOR-5635')

In [ ]:
pprint(jira.myself())